<a href="https://colab.research.google.com/github/MNoichl/tttms_public/blob/main/convert_latex_to_mathml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Google-Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install packages & latexml


In [ ]:
!pip install compress-pickle
!sudo apt-get install latexml
!sudo apt-get install libtext-unidecode-perl 
!apt-get update
!sudo apt-get install latexml
!latexmlmath \\frac{-b\\pm\\sqrt{b^2-4ac}}{2a}

# Load packages

In [ ]:
# parse arxiv data into formulas -> images,
# and fulltexts, and combine into one pickle-dataframe
import tqdm
import pandas as pd
import re
import os
import sys


import subprocess
import platform
import shutil

import numpy as np
from compress_pickle import dump, load
import zipfile

import seaborn as sns

# Download Tangent-Code & Data

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1NSx7sQw8Kk1zQl5aaLWftj8T9SHvaeCC/view?usp=sharing -O "TangentS.zip"
!unzip TangentS.zip

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1-5ZrFDj_6tff7tCRd8nqlVf6Ls-l8-x1/view?usp=sharing -O "formulas_to_process_v2.bz"
full_data = load("formulas_to_process_v2.bz")

In [ ]:
np.sum([len(x) for x in full_data['filtered_formulas'] if x != 'no formulas']) #check n_formulas

In [ ]:
full_data['formula_tuples'] = "no formulas"
full_data['actually_transformed_formulas'] = "no formulas"

Due to computational requirements conversion had to be split over several machines to be efficient. Below one such block, that starts conversion at a specific part.


In [ ]:
# %%capture
from xml.dom import minidom
from io import StringIO
from xml.dom.minidom import parseString
from TangentS.math_tan.math_extractor import MathExtractor
from TangentS.math_tan.symbol_tree import SymbolTree

starting_no = 0


for index, row in tqdm.tqdm_notebook(full_data[starting_no:].iterrows()):
  if isinstance(row['filtered_formulas'], list): 
    if len(row['filtered_formulas']) > 0:
      # paper_id = row['identification_string'].replace('.','_')
      
      try:
        with open('temporary_tex.tex', "w",encoding='utf-8') as text_file:
            text_file.write('\n \n \n'.join(row['filtered_formulas']))
        cmd = '( cmdpid=$BASHPID; (sleep 3; kill $cmdpid) & exec '+ 'latexml' + ' --preload=amsmath' + ' --preload=amsfonts' + ' --destination=temporary_xml.xml' +' temporary_tex.tex '+ ' --preload=mws.sty.ltxml ' + '--quiet ' + '--quiet' +')'
        cmd_output = !{cmd}
        cmd = '( cmdpid=$BASHPID; (sleep 3; kill $cmdpid) & exec '+ 'latexmlpost' + ' --contentmathml' + ' --destination=temporary_xml_2.xml' + ' temporary_xml.xml ' + '--quiet ' + '--quiet'+')'
        cmd_output = !{cmd}

        xmldoc = minidom.parse('temporary_xml_2.xml')
        
        xml_formula_list = xmldoc.getElementsByTagName('Math')
        # print(xml_formula_list)
        this_articles_tuples = []
        this_articles_formulas = []
        for i,this_xml_formula in enumerate(xml_formula_list):
          try:
            used_tex = this_xml_formula.getAttribute("tex")
            this_xml_formula = this_xml_formula.firstChild
            
            this_xml_formula.setAttribute("xmlns", "http://www.w3.org/1998/Math/MathML")
            out = StringIO()
            this_xml_formula.writexml(out)
            
            mathml = out.getvalue()
            
            mathml = mathml.replace('m:','')
            mathml = mathml.replace('<semantics>','').replace('<\semantics>','')
            mathml = '<?xml version="1.0" encoding="UTF-8"?>\n\n' + mathml
            cmml = MathExtractor.isolate_cmml(mathml)
            current_tree = MathExtractor.convert_to_semanticsymbol(cmml)
            temp = SymbolTree(current_tree)
            aaa = temp.get_pairs(window=2, eob=True)
            this_articles_tuples.append(aaa)
            this_articles_formulas.append(used_tex)
          except (KeyboardInterrupt, SystemExit):
            print("program stopped manually")
            raise
          except:
            pass

        full_data['formula_tuples'][index] = this_articles_tuples
        full_data['actually_transformed_formulas'][index] = this_articles_formulas

      except (KeyboardInterrupt, SystemExit):
        print("program stopped manually")
        raise
      except:
        pass
  if (index % 20 == 0):
    print('parsed until: ', index)
  if (index % 2000 == 0) and (index > starting_no+1):
    dump(full_data, "drive/MyDrive/combined_formula_parsing_results_v3/formula_tuple_list_"+ str(starting_no) +"_"+ str(index)+ ".bz")


